## Hashrate retrieval

This notebook retrieves hashrate from historical hashrate chart on bitinfocharts.com

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date, datetime
import pandas as pd

In [2]:
chrome_driver = webdriver.Chrome()

In [3]:
chrome_driver.get("https://bitinfocharts.com/comparison/zcash-hashrate.html")

Find the chart element

In [4]:
chart_element = chrome_driver.find_element_by_id("container")

Move mouse cursor to the hashrate point with first date. Offset is relative to the top left of the container.

In [5]:
(ActionChains(chrome_driver)
        .move_to_element_with_offset(chart_element, 56, 20)
        .perform())

In [6]:
bs = BeautifulSoup(chrome_driver.page_source, 'html.parser')

Get legend tag on the top left corner.

In [7]:
legend_tag = bs.select('div.dygraph-legend')

In [8]:
import re

In [9]:
result_list = []

In [10]:
first_point = re.sub("\xa0", " ", legend_tag[0].get_text()).split(': ')

In [11]:
result_list.append(first_point)

In [12]:
result_list[-1]

['2016/10/28', 'Zcash - Hashrate', '184.242k']

In [13]:
last_point_date = datetime.strptime(result_list[-1][0], '%Y/%m/%d').date()
today_date = date.today()
diff = (today_date - last_point_date).days

In [14]:
while(diff > 1):
    # move mouse cursor horizontally
    (ActionChains(chrome_driver)
        .move_by_offset(1.5, 0)
        .perform())
    bs = BeautifulSoup(chrome_driver.page_source, 'html.parser')
    legend_tag = bs.select('div.dygraph-legend')
    next_point = re.sub("\xa0", " ", legend_tag[0].get_text()).split(': ')
#     print(next_point)
    if next_point[0] != result_list[-1][0]:
        result_list.append(next_point)
    last_point_date = datetime.strptime(result_list[-1][0], '%Y/%m/%d').date()
    diff = (today_date - last_point_date).days

In [15]:
hashrate_df = pd.DataFrame.from_records(result_list, columns=['Date', 'Coin', 'Hashrate'])

In [16]:
hashrate_df['Date'] = hashrate_df['Date'].map(lambda x : datetime.strptime(x, '%Y/%m/%d').date())

In [17]:
hashrate_df['Unit'] = 'M'

In [18]:
hashrate_df['Hashrate'] = hashrate_df['Hashrate'].map(lambda x : float(x[:-1]) if x[-1]=='M' else float(x[:-1])/1000)

In [19]:
hashrate_df.head()

,Date,Coin,Hashrate,Unit
0,2016-10-28,Zcash - Hashrate,0.184242,M
1,2016-10-29,Zcash - Hashrate,1.659600,M
2,2016-10-30,Zcash - Hashrate,2.729600,M
3,2016-10-31,Zcash - Hashrate,2.909900,M
4,2016-11-01,Zcash - Hashrate,4.031100,M


In [20]:
hashrate_df.to_csv('./zcash_hashrate.csv', index=False)

In [21]:
chrome_driver.quit()